# Lab 23: Handling Twins and Close Relatives

## Overview

This notebook explores Bonsai v3's specialized handling of identical twins and other extremely close genetic relatives that present unique challenges for genetic genealogy. We'll examine how the `twins.py` module implements algorithms to detect twin-like patterns and ensures appropriate treatment in pedigree construction.

**Learning Objectives:**
- [List specific learning objectives for the lab]
- [Each objective should be measurable and aligned with the lab content]
- [For example: \\"Understand the architecture of the Bonsai relationship inference module\\"]

**Prerequisites:**
- [List any labs or concepts students should understand before starting]
- [For example: \\"Completion of Lab 3: IBD Formats\\"]

**Estimated completion time:** [e.g., 60-90 minutes]

In [ ]:
# Standard imports
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from IPython.display import display, HTML, Markdown
import inspect
import importlib

sys.path.append(os.path.dirname(os.getcwd()))

# Cross-compatibility setup
from scripts_support.lab_cross_compatibility import setup_environment, is_jupyterlite, save_results, save_plot

# Set up environment-specific paths
DATA_DIR, RESULTS_DIR = setup_environment()

# Set visualization styles
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_context("notebook")
sns.set_palette("colorblind")  # Improve accessibility with colorblind-friendly palette

# Configure plot defaults for better readability
plt.rcParams.update({
    'figure.figsize': (10, 6),
    'font.size': 12,
    'axes.labelsize': 12,
    'axes.titlesize': 14,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10
})

In [ ]:
# Setup Bonsai module paths
if not is_jupyterlite():
    # In local environment, add the utils directory to system path
    utils_dir = os.getenv('PROJECT_UTILS_DIR', os.path.join(os.path.dirname(DATA_DIR), 'utils'))
    bonsaitree_dir = os.path.join(utils_dir, 'bonsaitree')
    
    # Add to path if it exists and isn't already there
    if os.path.exists(bonsaitree_dir) and bonsaitree_dir not in sys.path:
        sys.path.append(bonsaitree_dir)
        print(f"Added {bonsaitree_dir} to sys.path")
else:
    # In JupyterLite, use a simplified approach
    print("⚠️ Running in JupyterLite: Some Bonsai functionality may be limited.")
    print("This notebook is primarily designed for local execution where the Bonsai codebase is available.")

In [ ]:
# Helper functions for exploring modules
def display_module_classes(module_name):
    """Display classes and their docstrings from a module"""
    try:
        # Import the module
        module = importlib.import_module(module_name)
        
        # Find all classes
        classes = inspect.getmembers(module, inspect.isclass)
        
        # Filter classes defined in this module (not imported)
        classes = [(name, cls) for name, cls in classes if cls.__module__ == module_name]
        
        if not classes:
            print(f"No classes found in module {module_name}")
            return
            
        # Print info for each class
        for name, cls in classes:
            display(Markdown(f"### Class: {name}"))
            
            # Get docstring
            doc = inspect.getdoc(cls)
            if doc:
                display(Markdown(f"**Documentation:**\
{doc}"))
            else:
                display(Markdown("*No documentation available*"))
            
            # Get methods
            methods = inspect.getmembers(cls, inspect.isfunction)
            public_methods = [(method_name, method) for method_name, method in methods 
                             if not method_name.startswith('_')]
            
            if public_methods:
                display(Markdown("**Public Methods:**"))
                for method_name, method in public_methods:
                    sig = inspect.signature(method)
                    display(Markdown(f"- `{method_name}{sig}`"))
            else:
                display(Markdown("*No public methods*"))
            
            display(Markdown("---"))
    except ImportError as e:
        print(f"Error importing module {module_name}: {e}")
    except Exception as e:
        print(f"Error processing module {module_name}: {e}")

def display_module_functions(module_name):
    """Display functions and their docstrings from a module"""
    try:
        # Import the module
        module = importlib.import_module(module_name)
        
        # Find all functions
        functions = inspect.getmembers(module, inspect.isfunction)
        
        # Filter functions defined in this module (not imported)
        functions = [(name, func) for name, func in functions if func.__module__ == module_name]
        
        if not functions:
            print(f"No functions found in module {module_name}")
            return
            
        # Filter public functions
        public_functions = [(name, func) for name, func in functions if not name.startswith('_')]
        
        if not public_functions:
            print(f"No public functions found in module {module_name}")
            return
            
        # Print info for each function
        for name, func in public_functions:                
            display(Markdown(f"### Function: {name}"))
            
            # Get signature
            sig = inspect.signature(func)
            display(Markdown(f"**Signature:** `{name}{sig}`"))
            
            # Get docstring
            doc = inspect.getdoc(func)
            if doc:
                display(Markdown(f"**Documentation:**\
{doc}"))
            else:
                display(Markdown("*No documentation available*"))
                
            display(Markdown("---"))
    except ImportError as e:
        print(f"Error importing module {module_name}: {e}")
    except Exception as e:
        print(f"Error processing module {module_name}: {e}")

def view_function_source(module_name, function_name):
    """Display the source code of a function"""
    try:
        # Import the module
        module = importlib.import_module(module_name)
        
        # Get the function
        func = getattr(module, function_name)
        
        # Get the source code
        source = inspect.getsource(func)
        
        # Print the source code with syntax highlighting
        display(Markdown(f"### Source code for `{function_name}`\
```python\
{source}\
```"))
    except ImportError as e:
        print(f"Error importing module {module_name}: {e}")
    except AttributeError:
        print(f"Function {function_name} not found in module {module_name}")
    except Exception as e:
        print(f"Error processing function {function_name}: {e}")

def view_class_source(module_name, class_name):
    """Display the source code of a class"""
    try:
        # Import the module
        module = importlib.import_module(module_name)
        
        # Get the class
        cls = getattr(module, class_name)
        
        # Get the source code
        source = inspect.getsource(cls)
        
        # Print the source code with syntax highlighting
        display(Markdown(f"### Source code for class `{class_name}`\
```python\
{source}\
```"))
    except ImportError as e:
        print(f"Error importing module {module_name}: {e}")
    except AttributeError:
        print(f"Class {class_name} not found in module {module_name}")
    except Exception as e:
        print(f"Error processing class {class_name}: {e}")

def explore_module(module_name):
    """Display a comprehensive overview of a module with classes and functions"""
    try:
        # Import the module
        module = importlib.import_module(module_name)
        
        # Module docstring
        doc = inspect.getdoc(module)
        display(Markdown(f"# Module: {module_name}"))
        
        if doc:
            display(Markdown(f"**Module Documentation:**\
{doc}"))
        else:
            display(Markdown("*No module documentation available*"))
            
        display(Markdown("---"))
        
        # Display classes
        display(Markdown("## Classes"))
        display_module_classes(module_name)
        
        # Display functions
        display(Markdown("## Functions"))
        display_module_functions(module_name)
        
    except ImportError as e:
        print(f"Error importing module {module_name}: {e}")
    except Exception as e:
        print(f"Error exploring module {module_name}: {e}")

## Check Bonsai Installation

Let's verify that the Bonsai v3 module is available for import:

In [ ]:
try:
    from bonsaitree import v3
    print("✅ Successfully imported Bonsai v3 module")
    
    # Print Bonsai version information if available
    if hasattr(v3, "__version__"):
        print(f"Bonsai v3 version: {v3.__version__}")
    
    # List key submodules
    print("\
Available Bonsai submodules:")
    for module_name in dir(v3):
        if not module_name.startswith("_") and not module_name.startswith("__"):
            print(f"- {module_name}")
except ImportError as e:
    print(f"❌ Failed to import Bonsai v3 module: {e}")
    print("This lab requires access to the Bonsai v3 codebase.")
    print("Make sure you've properly set up your environment with the Bonsai repository.")

## Introduction

[Provide a brief introduction to the lab topic and how it fits into the overall course structure.]

In this lab, we will explore [specific aspect of Bonsai v3] and its role in computational genetic genealogy. This topic is important because [explain relevance and importance].

**Key concepts we'll cover:**
- [Key concept 1]
- [Key concept 2]
- [Key concept 3]

## Part 1: [Section Title]

### Theory and Background

[Provide theoretical background for this section. Include diagrams or mathematical formulas if relevant.]

### Implementation in Bonsai v3

[Explain how this concept is implemented in the Bonsai v3 codebase, which modules and classes are involved, etc.]

### Exercise 1

[Provide a specific exercise for students to complete that reinforces the concepts from Part 1]

**Task:** [Clear description of what the student needs to do]

**Hint:** [Optional hint to guide students if they get stuck]

In [ ]:
# Exercise 1 code template
# Students can modify this code to complete the exercise

def exercise_1_function():
    # TODO: Implement the solution to Exercise 1
    pass

# Test your solution
# exercise_1_function()

In [ ]:
# Demonstration code for Part 1
# Include comments explaining what each part does

## Part 2: [Section Title]

### Theory and Background

[Provide theoretical background for this section.]

### Implementation in Bonsai v3

[Explain how this concept is implemented in the Bonsai v3 codebase.]

### Exercise 2

[Provide a specific exercise for students to complete that reinforces the concepts from Part 2]

**Task:** [Clear description of what the student needs to do]

**Hint:** [Optional hint to guide students if they get stuck]

In [ ]:
# Exercise 2 code template
# Students can modify this code to complete the exercise

def exercise_2_function():
    # TODO: Implement the solution to Exercise 2
    pass

# Test your solution
# exercise_2_function()

In [ ]:
# Demonstration code for Part 2
# Include visualization if appropriate

# Example visualization code:
# plt.figure(figsize=(10, 6))
# x = np.linspace(0, 10, 100)
# y = np.sin(x)
# plt.plot(x, y)
# plt.title('Example Visualization')
# plt.xlabel('X axis')
# plt.ylabel('Y axis')
# plt.grid(True)
# plt.show()

## Part 3: [Section Title]

### Theory and Background

[Provide theoretical background for this section.]

### Implementation in Bonsai v3

[Explain how this concept is implemented in the Bonsai v3 codebase.]

### Exercise 3

[Provide a specific exercise for students to complete that reinforces the concepts from Part 3 and potentially integrates concepts from Parts 1 and 2]

**Task:** [Clear description of what the student needs to do]

**Hint:** [Optional hint to guide students if they get stuck]

In [ ]:
# Exercise 3 code template
# Students can modify this code to complete the exercise

def exercise_3_function():
    # TODO: Implement the solution to Exercise 3
    pass

# Test your solution
# exercise_3_function()

## Real-World Application

[Provide a concrete example of how the concepts from this lab apply to real-world genetic genealogy scenarios. This helps students understand the practical relevance of what they're learning.]

### Case Study: [Title]

[Describe a realistic scenario where these techniques would be applied]

In [ ]:
# Case study demonstration code
# Include realistic data examples if possible

In [ ]:
# Demonstration code for Part 3
# Include an example that ties together concepts from previous sections

## Self-Assessment Questions

Test your understanding with these questions:

1. [Question 1]
2. [Question 2]
3. [Question 3]

*Answers to self-assessment questions can be found at the end of the lab document.*

## Summary

In this lab, we explored [summary of what was covered]. Key takeaways include:

1. [Takeaway 1]
2. [Takeaway 2]
3. [Takeaway 3]

### Connections to Other Labs

The concepts covered in this lab connect to:
- **Lab X:** [Brief explanation of connection]
- **Lab Y:** [Brief explanation of connection]

### Further Reading

To deepen your understanding of these topics, consider exploring:

- [Reference 1]
- [Reference 2]
- [Reference 3]

---

## Answer Key (for instructors)

### Exercise 1
```python
# Solution code for Exercise 1
```

### Exercise 2
```python
# Solution code for Exercise 2
```

### Exercise 3
```python
# Solution code for Exercise 3
```

### Self-Assessment Answers

1. [Answer to Question 1]
2. [Answer to Question 2]
3. [Answer to Question 3]

In [ ]:
# Optional: Convert this notebook to PDF
# Uncomment and run this cell if you want to generate a PDF version

# !jupyter nbconvert --to pdf "$(basename \\"$PWD\\").ipynb"